In [42]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import yfinance as yf
import pandas as pd
import os
import json
import gzip
import re

In [2]:
data = '../data/ratings/Appliances.csv'

col_names = ['Item ID', 'User ID', 'Rating', 'Timestamp']

df = pd.read_csv(data, names = col_names)

In [3]:
df

,Item ID,User ID,Rating,Timestamp
0,1118461304,A3NHUQ33CFH3VM,5.0,1385510400
1,1118461304,A3SK6VNBQDNBJE,5.0,1383264000
2,1118461304,A3SOFHUR27FO3K,5.0,1381363200
3,1118461304,A1HOG1PYCAE157,5.0,1381276800
4,1118461304,A26JGAM6GZMM4V,5.0,1378512000
...,...,...,...,...
602772,B01HJH2PY0,A24A9P4F2SLTK5,5.0,1502323200
602773,B01HJHHEA0,A2JCB4KHBWEELW,2.0,1533081600
602774,B01HJHHEA0,A1LDYYVTLPP2Z5,5.0,1523577600
602775,B01HJH92JQ,AP1M5O06IOYZ7,1.0,1521763200


In [4]:
times = df['Timestamp']

print(len(times))

for i in range(len(times)):
    times[i] = datetime.datetime.fromtimestamp(times[i]) # This will give a warning, but does do what I need it to
    # times.replace(times[i], datetime.datetime.fromtimestamp(times[i]))

602777


/tmp/ipykernel_46351/2913038016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  times[i] = datetime.datetime.fromtimestamp(times[i]) # This will give a warning, but does do what I need it to


In [5]:
times

0         2013-11-26 16:00:00
1         2013-10-31 17:00:00
2         2013-10-09 17:00:00
3         2013-10-08 17:00:00
4         2013-09-06 17:00:00
                 ...         
602772    2017-08-09 17:00:00
602773    2018-07-31 17:00:00
602774    2018-04-12 17:00:00
602775    2018-03-22 17:00:00
602776    2018-01-07 16:00:00
Name: Timestamp, Length: 602777, dtype: object

In [6]:
df = df.drop(labels = 'Timestamp', axis = 1)
df = pd.concat([df,times], axis = 1)
df

,Item ID,User ID,Rating,Timestamp
0,1118461304,A3NHUQ33CFH3VM,5.0,2013-11-26 16:00:00
1,1118461304,A3SK6VNBQDNBJE,5.0,2013-10-31 17:00:00
2,1118461304,A3SOFHUR27FO3K,5.0,2013-10-09 17:00:00
3,1118461304,A1HOG1PYCAE157,5.0,2013-10-08 17:00:00
4,1118461304,A26JGAM6GZMM4V,5.0,2013-09-06 17:00:00
...,...,...,...,...
602772,B01HJH2PY0,A24A9P4F2SLTK5,5.0,2017-08-09 17:00:00
602773,B01HJHHEA0,A2JCB4KHBWEELW,2.0,2018-07-31 17:00:00
602774,B01HJHHEA0,A1LDYYVTLPP2Z5,5.0,2018-04-12 17:00:00
602775,B01HJH92JQ,AP1M5O06IOYZ7,1.0,2018-03-22 17:00:00


In [33]:
def get_five_core_data(directory):
    five_core = {}
    
    # Iterate through the data in the given directory
    for subdir, dirs, files in os.walk(directory):

        # For each file, get the filename and extract the JSON
        for file in files:
            filename = str(os.path.join(subdir, file))
            five_core_data = []
            # Use gzip to open the .gz and extract each line using the normal file package
            with gzip.open(filename, "rb") as f:
                for line in f:
                    line = line.decode(encoding='UTF-8')
                    json_data = json.loads(line)
                    five_core_data.append(json_data)
            # Add data to the dict
            five_core[filename] = five_core_data
           
    # Return the dict of complete ratings from each file
    return five_core

def get_ratings_data(directory):
    ratings = {}
    col_names = ['Item ID', 'User ID', 'Rating', 'Timestamp']
    
    # Iterate through the data in the given directory
    for subdir, dirs, files in os.walk(directory):

        # For each file, get the filename and extract the DF
        for file in files:
            filename = str(os.path.join(subdir, file))
            # Create a DF of the data and add it to the dict
            df = pd.read_csv(filename, names = col_names)
            ratings[filename] = df
           
    # Return the dict of complete ratings from each file
    return ratings

In [34]:
five_core = get_five_core_data('../data/5-core/')
five_core

{'../data/5-core/Magazine_Subscriptions_5.json.gz': [{'overall': 4.0,
   'verified': True,
   'reviewTime': '02 26, 2014',
   'reviewerID': 'A5QQOOZJOVPSF',
   'asin': 'B00005N7P0',
   'reviewerName': 'John L. Mehlmauer',
   'reviewText': "I'm old, and so is my computer.  Any advice that can help me maximize my computer perfomance is very welcome.  MaximumPC has some good tips on computer parts, vendors, and usefull tests",
   'summary': 'Cheapskates guide',
   'unixReviewTime': 1393372800},
  {'overall': 5.0,
   'vote': '3',
   'verified': False,
   'reviewTime': '03 6, 2004',
   'reviewerID': 'A5RHZE7B8SV5Q',
   'asin': 'B00005N7PS',
   'reviewerName': 'gorillazfan249',
   'reviewText': "There's nothing to say, but if you want a REAL men's magazine, this is it. They have great articles and stories, and I love the bits of knowledge that starts the Know & Tell section, and I love the last page, which has an interview with a celebrity. Just get this magazine and forget all the other mat

In [35]:
for i in five_core:
    print(len(i))
five_core.keys()

47
35


dict_keys(['../data/5-core/Magazine_Subscriptions_5.json.gz', '../data/5-core/Appliances_5.json.gz'])

In [21]:
# Find most frequently used words
# get_five_core_data for a relevant dataset -> may want to enable singular datasets
# Get keys of 5 core data
# Change keys to a list
# Get 'reviewText'
# Get counts of each word in reviewText and put into dict (there is a lab on this)
# Sort dict by values
# Show 10 most commonly used words for that dataset
five_core[list(five_core.keys())[0]][0].keys()

dict_keys(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime'])

In [57]:
def get_most_used_words(directory):
    five_core = get_five_core_data(directory)
    five_core_keys = list(five_core.keys())
    counts = {}
    
    for key in five_core_keys:
        for i in range(len(five_core[key])):
            review_text = five_core[key][i].get('reviewText')
            if review_text is not None:
                review_words = review_text.split()

                for word in review_words:
                    lower_word = re.sub(r'[^\w\s]', '', word.lower())
                    counts[lower_word] = counts.get(lower_word, 0) + 1
                    
    counts_list = sorted(counts.items(), key=lambda x:x[1])
    
    return counts_list[-10:]

test = get_most_used_words('../data/5-core/')

In [58]:
test

[('this', 8314),
 ('in', 9825),
 ('you', 9991),
 ('it', 13299),
 ('of', 14795),
 ('a', 18373),
 ('to', 22614),
 ('i', 22681),
 ('and', 24479),
 ('the', 49519)]

In [11]:
# Pie chart of most frequently used words

In [12]:
# Average rating vs Amazon stock price

In [13]:
# Average rating over time
# Stock price over time

In [14]:
# Linear regression of stock price over time